In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pickle
import json

In [2]:
import optuna
from sklearn.metrics import log_loss
from catboost import CatBoostRegressor

In [3]:
import warnings
warnings.filterwarnings("ignore")

In [4]:
pd.set_option('display.max_columns', None)

In [5]:
def MAE(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs(y_true - y_pred))

In [6]:
def MSE(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean((y_true - y_pred)**2)

In [7]:
def RMSE(y_true, y_pred):
    return np.sqrt(MSE(y_true, y_pred))

In [8]:
def R2(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return(1-np.sum((y_true - y_pred)**2)/np.sum((y_true - y_true.mean())**2))

In [9]:

def objective(trial, df, features, split_dict):
    param_grid = {
        "n_estimators": trial.suggest_int("n_estimators", 1000, 5000),
        "loss_function": trial.suggest_categorical("loss_function", ["RMSE", "MAE"]),
        "learning_rate": trial.suggest_loguniform("learning_rate", 1e-5, 1e0),
        "l2_leaf_reg": trial.suggest_loguniform("l2_leaf_reg", 1e-2, 1e0),
        "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.01, 0.1),
        "depth": trial.suggest_int("depth", 1, 10),
        "boosting_type": trial.suggest_categorical("boosting_type", ["Ordered", "Plain"]),
        "bootstrap_type": trial.suggest_categorical("bootstrap_type", ["Bayesian", "Bernoulli", "MVS"]),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 2, 20),
        "one_hot_max_size": trial.suggest_int("one_hot_max_size", 2, 20)
    }

    cv_scores = []
    
    for year, (train_idx, valid_idx) in split_dict.items():
        X_train, X_valid = df.loc[df.Field_ID.isin(train_idx), features].values, df.loc[df.Field_ID.isin(valid_idx), features].values
        y_train, y_valid = df.loc[df.Field_ID.isin(train_idx)].Yield.values, df.loc[df.Field_ID.isin(valid_idx)].Yield.values

        model = CatBoostRegressor(**param_grid, random_state=42)
        model.fit(
              X_train,
              y_train,
              eval_set=[(X_valid, y_valid)],
              verbose=False
          )
        preds = model.predict(X_valid)
        cv_scores.append(RMSE(preds, y_valid))
        print('{}, RMSE: {:.3f}'.format(year, np.mean(cv_scores)))
    
    print()

    return np.mean(cv_scores)

# Data

In [10]:
data = pd.read_csv('./csv/data_norm.csv')

In [11]:
# load dict of ids for splitting 
with open("./csv/split.json", "r") as read_file:
    split = json.load(read_file)
split.keys()

dict_keys(['train_ids', 'test_ids', 'year_split'])

In [12]:
# load dict of features groups
with open("./csv/feature_selection.json", "r") as read_file:
    features_dict = json.load(read_file)
features_dict.keys()

dict_keys(['features', 'RMSE', 'R2', 'score'])

In [13]:
features = features_dict['features']

In [14]:
len(features)

357

In [15]:
data = data.loc[data.Field_ID.isin(split['train_ids'])]

In [16]:
box = data.copy()
box[features] = data[features].fillna(data[features].mean())

In [17]:
study = optuna.create_study(direction="minimize", study_name="Catboost Regressor")
func = lambda trial: objective(trial, box, features, split['year_split'])
study.optimize(func, n_trials=20, timeout=60*60) ## one hour

[I 2022-04-20 19:30:28,218] A new study created in memory with name: Catboost Regressor


2019, RMSE: 1.795
2018, RMSE: 1.761
2017, RMSE: 1.700


[I 2022-04-20 19:38:43,392] Trial 0 finished with value: 1.6505924162300438 and parameters: {'n_estimators': 1211, 'loss_function': 'MAE', 'learning_rate': 0.04198105847130205, 'l2_leaf_reg': 0.07252031790520193, 'colsample_bylevel': 0.050981895111642556, 'depth': 9, 'boosting_type': 'Plain', 'bootstrap_type': 'Bayesian', 'min_data_in_leaf': 19, 'one_hot_max_size': 18}. Best is trial 0 with value: 1.6505924162300438.


2016, RMSE: 1.651

2019, RMSE: 1.803
2018, RMSE: 1.757
2017, RMSE: 1.693


[I 2022-04-20 19:49:57,143] Trial 1 finished with value: 1.6483371428488547 and parameters: {'n_estimators': 3011, 'loss_function': 'MAE', 'learning_rate': 0.0008008515709068984, 'l2_leaf_reg': 0.013948584223279555, 'colsample_bylevel': 0.08155732112121207, 'depth': 7, 'boosting_type': 'Plain', 'bootstrap_type': 'MVS', 'min_data_in_leaf': 17, 'one_hot_max_size': 8}. Best is trial 1 with value: 1.6483371428488547.


...

2016, RMSE: 1.622

2019, RMSE: 1.744
2018, RMSE: 1.716
2017, RMSE: 1.661


[I 2022-04-20 20:29:04,119] Trial 9 finished with value: 1.6259998129555566 and parameters: {'n_estimators': 4405, 'loss_function': 'RMSE', 'learning_rate': 0.001033014144330719, 'l2_leaf_reg': 0.10000168578567854, 'colsample_bylevel': 0.0928655719506299, 'depth': 2, 'boosting_type': 'Ordered', 'bootstrap_type': 'MVS', 'min_data_in_leaf': 18, 'one_hot_max_size': 6}. Best is trial 6 with value: 1.6090459926264473.


2016, RMSE: 1.626

2019, RMSE: 1.903
2018, RMSE: 1.864
2017, RMSE: 1.793


[I 2022-04-20 20:39:36,500] Trial 10 finished with value: 1.787545798691918 and parameters: {'n_estimators': 1882, 'loss_function': 'RMSE', 'learning_rate': 2.3676073811144295e-05, 'l2_leaf_reg': 0.2823714441507256, 'colsample_bylevel': 0.03011192169295769, 'depth': 10, 'boosting_type': 'Ordered', 'bootstrap_type': 'Bernoulli', 'min_data_in_leaf': 12, 'one_hot_max_size': 2}. Best is trial 6 with value: 1.6090459926264473.


2016, RMSE: 1.788



In [18]:

print(f"\tBest value (rmse): {study.best_value:.5f}")
print(f"\tBest params:")

for key, value in study.best_params.items():
    print(f"\t\t{key}: {value}")

	Best value (rmse): 1.60905
	Best params:
		n_estimators: 3372
		loss_function: RMSE
		learning_rate: 0.018124470795854327
		l2_leaf_reg: 0.5992324216820445
		colsample_bylevel: 0.06584578982175233
		depth: 7
		boosting_type: Ordered
		bootstrap_type: MVS
		min_data_in_leaf: 2
		one_hot_max_size: 3


In [21]:
with open('./csv/catboost_optuna.json', 'w') as file:
    json.dump(study.best_params, file)